In [ ]:
import sys

In [ ]:
f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

In [ ]:
f = open('labels.txt')
raw_labels = f.readlines()
f.close()

In [ ]:
tokens = list(map(lambda x: set(x.split(" ")), raw_reviews ))

In [ ]:
vocab = set()
for sent in tokens:
    for word in sent:
        if len(word) >0 :
            vocab.add(word)
vocab = list(vocab)

In [ ]:
word2index = {}
for i,word in enumerate(vocab):
    word2index[word] = i

In [ ]:
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

In [ ]:
target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)
        

In [ ]:
import numpy as np
np.random.seed(1)

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
alpha, iterations = (0.01 , 2)
hidden_size = 100

In [ ]:
weights_0_1 = 0.02*np.random.random((len(vocab),hidden_size)) - 0.01
weights_1_2 = 0.02*np.random.random((hidden_size, 1)) - 0.01
correct, total = (0,0)

In [ ]:
for iter in range(iterations):
    
    for i in range(len(input_dataset) - 1000):
        x,y = (input_dataset[i], target_dataset[i])
        layer_1 = sigmoid(np.sum((weights_0_1[x]),axis=0))
        layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
                          
        layer_2_delta = layer_2 - y
        layer_1_delta = layer_2_delta.dot(weights_1_2.T)
                          
        weights_0_1[x] -= layer_1_delta*alpha
        weights_1_2 -= np.outer(layer_1,layer_2_delta) * alpha
                          
        if(np.abs(layer_2_delta) < 0.5):
            correct +=1
        total +=1
        if(i % 10 == 9):
            progress = str(i/float(len(input_dataset)))
            sys.stdout.write('\rIter:' + str(iter) \
                            +' Progress:' + progress[2:4] \
                            + '.' + progress[4:6]\
                            +'% Training Accuracy:' \
                            + str(correct/float(total)) + "%" )
    print()
    
correct,total = (0,0)

for i in range(len(input_dataset) - 1000, len(input_dataset)):
    x = input_dataset[i]
    y = target_dataset[i]
    
    layer_1 = sigmoid(np.sum(weights_0_1[x], axis=0))
    layer_2 = sigmoid(np.dot(layer_1, weights_1_2))
    
    if (np.abs(layer_2-y) < 0.5):
        correct +=1
    total +=1 
print("Test Accuracy:" + str(correct/ float(total)))
                          
                          

In [ ]:
from collections import Counter
import math

In [ ]:
def similar(target='beautiful'):
    target_index = word2index[target]
    scores = Counter()
    for word,index in word2index.items():
        raw_difference = weights_0_1[index] - weights_0_1[target_index]
        squared_difference = raw_difference**2
        scores[word] = -math.sqrt(sum(squared_difference))
    for w in scores.most_common(20):
        print(w)